Code to evaluate the energy and ergotropy 

In [ ]:
from qutip import *
import numpy as np
from qutip.qip.operations import gates as gt
import time
import matplotlib.pyplot as plt
import pandas as pd
from multiprocessing import Pool


def measurements(rho):
    sx = (sigmax() * rho).tr()
    sy = (sigmay() * rho).tr()
    sz = (sigmaz() * rho).tr()
    rmod = np.sqrt(sx**2 +sy**2 + sz**2)
    ergotropy = 0.5 * (rmod + sz)
    energy = 0.5 * (1 + sz)
    ergotropy_i = 0.5 * (abs(sz) + sz)
    ergotropy_c = ergotropy - ergotropy_i
    return np.real(energy), np.real(ergotropy) # , np.real(ergotropy_i), np.real(ergotropy_c)

def G_calculation(dt, gamma):
    G = np.sqrt(gamma / dt)
    return G

def markovian_unitary(dt, F, G, g):
    Hamdrive = tensor(identity(2), F * sigmax(), identity(2))
    HamIntCHBatt = tensor(identity(2), g * (tensor(sigmap(), sigmam()) + tensor(sigmam(), sigmap())))
    HamIntChCM = tensor( G * (tensor(sigmap(), sigmam()) + tensor(sigmam(), sigmap())), identity(2))
    return (-1j * dt * (Hamdrive + HamIntCHBatt + HamIntChCM)).expm()

def states(beta_cm, beta_charger, beta_battery):
    RhoCharger = 0.5 * (identity(2) - np.tanh(0.5 * beta_charger) * sigmaz())
    RhoBattery = 0.5 * (identity(2) - np.tanh(0.5 * beta_battery) * sigmaz())
    RhoCM = 0.5*(identity(2) - np.tanh(0.5 * beta_cm) * sigmaz())
    RhoComp = tensor(RhoCM, RhoCharger, RhoBattery)
    return RhoComp, RhoCM


def evolution(g, p, F, dt, gamma=1, beta_cm=100, beta_charger=100, beta_battery=100):
    ergotropy, energy= [],[]
    # ergotropy_i, ergotropy_c= [], []
    # p=np.exp(-p*dt) # uncomment if in the strongly non-Markovian continuous-time limit
    rho_composite, rho_cm = states(beta_cm, beta_charger, beta_battery)
    G = G_calculation(dt, gamma)
    Um = markovian_unitary(dt, F, G, g)
    incoherent_swap = tensor(gt.swap(),identity(2),identity(2))
    stop=False
    for i in range(10**(int(-np.log10(dt))+3)):
        if i % 10**(int(-np.log10(dt))) == 0:
            *x, = measurements(rho_composite.ptrace(2))
            energy.append(x[0])
            ergotropy.append(x[1])
            # ergotropy_i.append(x[2])
            # ergotropy_c.append(x[3])
        rho_composite = (Um * rho_composite * Um.dag())
        rho_composite = tensor(rho_cm, rho_composite)
        rho_composite = (1-p)*rho_composite + (p)*incoherent_swap*rho_composite*incoherent_swap
        rho_composite = rho_composite.ptrace([0,2,3])
    i=1
    while stop==False:
        if i% 10**(int(-np.log10(dt))) == 0:
            *x ,= measurements(rho_composite.ptrace(2))
            energy.append(x[0])
            ergotropy.append(x[1])
            # ergotropy_i.append(x[2])
            # ergotropy_c.append(x[3])
            lenght = int(len(energy) / 5)
            check = np.full([lenght], energy[-1])
            if np.array_equal(np.round(check,5),np.round(energy[-lenght:],5))==True:
                stop=True
        rho_composite = (Um * rho_composite * Um.dag())
        rho_composite = tensor(rho_cm, rho_composite)
        rho_composite = (1-p)*rho_composite + (p)*incoherent_swap*rho_composite*incoherent_swap
        rho_composite = rho_composite.ptrace([0,2,3])
        i += 1
    return energy, ergotropy
    # return energy, ergotropy # , ergotropy_i, ergotropy_c




code to evaluate the geometrical measure for both qubit

In [ ]:
from qutip import *
import numpy as np
from qutip.qip.operations import gates as gt
from tqdm.notebook import trange, tqdm
import time
import matplotlib.pyplot as plt
import pandas as pd
    
def G_calculation(dt, gamma):
    G = np.sqrt(gamma / dt)
    return G

def markovian_unitary(dt, F, G, g):
    Hamdrive = tensor(identity(2), F * sigmax(), identity(2))
    HamIntCHBatt = tensor(identity(2), g * (tensor(sigmap(), sigmam()) + tensor(sigmam(), sigmap())))
    HamIntChCM = tensor( G * (tensor(sigmap(), sigmam()) + tensor(sigmam(), sigmap())), identity(2))
    return (-1j * dt * (Hamdrive + HamIntCHBatt + HamIntChCM)).expm()

def generators():
    l0=Qobj(0.5 * np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]]),dims=[[2,2],[2,2]])
    l1=Qobj(1/np.sqrt(2) *np.array([[0,1,0,0],[1,0,0,0],[0,0,0,0],[0,0,0,0]]),dims=[[2,2],[2,2]])
    l2=Qobj(1/np.sqrt(2) *np.array([[0,1j,0,0],[-1j,0,0,0],[0,0,0,0],[0,0,0,0]]),dims=[[2,2],[2,2]])
    l3=Qobj(1/np.sqrt(2) *np.array([[1,0,0,0],[0,-1,0,0],[0,0,0,0],[0,0,0,0]]),dims=[[2,2],[2,2]])
    l4=Qobj(1/np.sqrt(2) *np.array([[0,0,1,0],[0,0,0,0],[1,0,0,0],[0,0,0,0]]),dims=[[2,2],[2,2]])
    l5=Qobj(1/np.sqrt(2) *np.array([[0,0,-1j,0],[0,0,0,0],[1j,0,0,0],[0,0,0,0]]),dims=[[2,2],[2,2]])
    l6=Qobj(1/np.sqrt(2) *np.array([[0,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,0]]),dims=[[2,2],[2,2]])
    l7=Qobj(1/np.sqrt(2) *np.array([[0,0,0,0],[0,0,-1j,0],[0,1j,0,0],[0,0,0,0]]),dims=[[2,2],[2,2]])
    l8=Qobj((1/np.sqrt(6)) * np.array([[1,0,0,0],[0,1,0,0],[0,0,-2,0],[0,0,0,0]]),dims=[[2,2],[2,2]])
    l9=Qobj(1/np.sqrt(2) *np.array([[0,0,0,1],[0,0,0,0],[0,0,0,0],[1,0,0,0]]),dims=[[2,2],[2,2]])
    l10=Qobj(1/np.sqrt(2) *np.array([[0,0,0,-1j],[0,0,0,0],[0,0,0,0],[1j,0,0,0]]),dims=[[2,2],[2,2]])
    l11=Qobj(1/np.sqrt(2) *np.array([[0,0,0,0],[0,0,0,1],[0,0,0,0],[0,1,0,0]]),dims=[[2,2],[2,2]])
    l12=Qobj(1/np.sqrt(2) *np.array([[0,0,0,0],[0,0,0,-1j],[0,0,0,0],[0,1j,0,0]]),dims=[[2,2],[2,2]])
    l13=Qobj(1/np.sqrt(2) *np.array([[0,0,0,0],[0,0,0,0],[0,0,0,1],[0,0,1,0]]),dims=[[2,2],[2,2]])
    l14=Qobj(1/np.sqrt(2) *np.array([[0,0,0,0],[0,0,0,0],[0,0,0,-1j],[0,0,1j,0]]),dims=[[2,2],[2,2]])
    l15=Qobj((1/np.sqrt(12)) * np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,-3]]),dims=[[2,2],[2,2]])
    return l0,l1,l2,l3,l4,l5,l6,l7,l8,l9,l10,l11,l12,l13,l14,l15

def matrix_shape(args,gen):
    M = []
    for i in range(len(gen)):
        for j in range(len(args)):
            M.append((gen[i] * args[j]).tr())
    Matrix = np.reshape(np.array(M),[len(gen),len(args)])
    return Matrix

def evolution(g, p, F=1, gamma=1, beta_cm=100, dt=0.1, step=5000,frequence=10):
    ln, lt, d1, rho_composite, determinant= [], [], [], [], []
    measure = 0
    # p=np.exp(-p*dt)
    gen = generators()
    rho_cm = 0.5*(identity(2) - np.tanh(0.5 * beta_cm) * sigmaz())
    for i in range(len(gen)):
        rho_composite.append(tensor(rho_cm, gen[i]))
    G = G_calculation(dt, gamma)
    Um = markovian_unitary(dt, F, G, g)
    incoherent_swap = tensor(gt.swap(),identity(2),identity(2))
    for i in trange(step):
        if i%frequence==0:
            for j in range(16):
                l = rho_composite[j].ptrace([1,2])
                ln.append(l)
            lt.append(ln)
            ln = []
        for j in range(16):
            rho_composite[j] = (Um * rho_composite[j] * Um.dag())
            rho_composite[j] = tensor(rho_cm, rho_composite[j])
            rho_composite[j] = (1-p)*rho_composite[j] + (p)*incoherent_swap*rho_composite[j]*incoherent_swap
            rho_composite[j] = rho_composite[j].ptrace([0,2,3])
    for i in range(len(lt)):
        determinant.append(np.linalg.det(matrix_shape(lt[i],gen)))
    for i in range(len(determinant)-1):
        d1.append(determinant[i+1]-determinant[i])
        if np.sign(d1[-1])== 1:
            measure = measure + d1[i]
    i = step
    stop = False
    pbar= tqdm(desc='last cycle', total=1)
    while stop == False:
        if i%frequence==0:
            for j in range(16):
                l = rho_composite[j].ptrace([1,2])
                ln.append(l)
            determinant.append(np.linalg.det(matrix_shape(ln, gen)))
            ln = []
            d1.append(determinant[-1]- determinant[-2])
            if np.sign(d1[-1]) == 1:
                measure = measure + d1[-1]
            lenght = int(len(determinant) / 10)
            check = np.full([lenght], determinant[-1])
            if np.array_equal(np.round(check, 15), np.round(determinant[-lenght:], 15)) == True:
                stop = True
        for j in range(16):
            rho_composite[j] = (Um * rho_composite[j] * Um.dag())
            rho_composite[j] = tensor(rho_cm, rho_composite[j])
            rho_composite[j] = (1 - p) * rho_composite[j] + (p) * incoherent_swap * rho_composite[j] * incoherent_swap
            rho_composite[j] = rho_composite[j].ptrace([0, 2, 3])
        i += 1
        pbar.update(1)
    return determinant, measure

Code to evaluate the geometric measure for the battery

In [ ]:
from qutip import *
import numpy as np
from qutip.qip.operations import gates as gt
from tqdm.notebook import trange, tqdm
import time
import matplotlib.pyplot as plt
import psutil
import line_profiler
import memory_profiler

    
def G_calculation(dt, gamma):
    G = np.sqrt(gamma / dt)
    return G

def markovian_unitary(dt, F, G, g):
    Hamdrive = tensor(identity(2), F * sigmax(), identity(2))
    HamIntCHBatt = tensor(identity(2), g * (tensor(sigmap(), sigmam()) + tensor(sigmam(), sigmap())))
    HamIntChCM = tensor( G * (tensor(sigmap(), sigmam()) + tensor(sigmam(), sigmap())), identity(2))
    return (-1j * dt * (Hamdrive + HamIntCHBatt + HamIntChCM)).expm()

def generators():
    return 1/np.sqrt(2) * identity(2),1/np.sqrt(2) * sigmax(),1/np.sqrt(2) * sigmay(),1/np.sqrt(2) * sigmaz()

def matrix_shape(args,gen):
    M = []
    for i in range(len(gen)):
        for j in range(len(args)):
            M.append((gen[i] * args[j]).tr())
    Matrix = np.reshape(np.array(M),[len(gen),len(args)])
    return Matrix
               
def evolution(g, p, F, gamma=1, beta_cm=100, beta_charger=100, dt=0.01, step=200,frequence=1):
    ln, lt, d1, rho_composite, determinant= [], [], [], [], []
    measure = 0
    # p=np.exp(-p*dt)
    gen = generators()
    rho_cm = 0.5*(identity(2) - np.tanh(0.5 * beta_cm) * sigmaz())
    rho_charger = 0.5*(identity(2) - np.tanh(0.5 * beta_charger) * sigmaz())
    for i in range(3):
        rho_composite.append(tensor(rho_cm, rho_charger, gen[i+1]))
    G = G_calculation(dt, gamma)
    Um = markovian_unitary(dt, F, G, g)
    Umd = markovian_unitary(-dt, F, G, g)
    incoherent_swap = tensor(gt.swap(),identity(2), identity(2))
    pbar = tqdm(desc='time', total=1)
    for i in range(step):
        if i%frequence==0:
            for j in range(3):
                l = rho_composite[j].ptrace(2)
                ln.append(l)
            lt.append(ln)
            ln = []
            pbar.update(1)
        for j in range(3):
            rho_composite[j] = (Um * rho_composite[j] * Umd)
            rho_composite[j] = tensor(rho_cm, rho_composite[j])
            rho_composite[j] = (1-p)*rho_composite[j] + (p)*incoherent_swap*rho_composite[j]*incoherent_swap
            rho_composite[j] = rho_composite[j].ptrace([0,2,3])
    for i in range(len(lt)):
        determinant.append(np.real(np.linalg.det(matrix_shape(lt[i],gen[1:]))))
        pbar.update(1)
    for i in range(len(determinant)-1):
        d1.append(determinant[i+1] - determinant[i])
        if np.sign(d1[-1])== 1:
            measure = measure + d1[i]
    stop = False
    i=1
    while stop==False:
        if i%frequence==0:
            for j in range(3):
                l = rho_composite[j].ptrace(2)
                ln.append(l)
            determinant.append(np.real(np.linalg.det(matrix_shape(ln,gen[1:]))))
            d1.append(determinant[-1] - determinant[-2])
            ln = []
            if np.sign(d1[-1])==1:
                measure = measure + d1[-1]
            lenght = int(len(determinant) / 20)
            check = np.full([lenght], np.round(determinant[-1],22))
            if np.array_equal(check,np.round(determinant[-lenght:],22))==True:
                stop=True
            pbar.update(1)
        for j in range(3):
            rho_composite[j] = (Um * rho_composite[j] * Umd)
            rho_composite[j] = tensor(rho_cm, rho_composite[j])
            rho_composite[j] = (1-p)*rho_composite[j] + (p)*incoherent_swap*rho_composite[j]*incoherent_swap
            rho_composite[j] = rho_composite[j].ptrace([0,2,3])
        i +=1
    return determinant, measure, lt
